In [1]:
!pip install autokeras

     |████████████████████████████████| 166 kB 5.4 MB/s 
     |████████████████████████████████| 454.4 MB 23 kB/s 
     |████████████████████████████████| 97 kB 6.6 MB/s 
     |████████████████████████████████| 462 kB 40.7 MB/s 
     |████████████████████████████████| 4.0 MB 37.7 MB/s 
     |████████████████████████████████| 1.2 MB 35.5 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68722 sha256=92ab6a9b9a8ff34a4e42d46dfb7e31143712b13874ebfc002f4d0e1ba2acbb08
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.42.0
    Uninstalling grpcio-1.42.0:
      Successfully uninstalled grpcio-1.42.0
  A

In [15]:
%tensorflow_version 2.x
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2Dt
# from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import metrics
from scipy.stats import pearsonr
import autokeras as ak
from time import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data set

In [4]:
training_impute_median = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/training_impute_median.csv')
training_impute_median_norm = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/training_impute_median_norm.csv')
testing_impute_median = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/testing_impute_median.csv')
testing_impute_median_norm = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/testing_impute_median_norm.csv')
validation_impute_median = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/validation_impute_median.csv')
validation_impute_median_norm = pd.read_csv('/content/drive/MyDrive/ML Project/Data for modeling/median imputer/validation_impute_median_norm.csv')

In [5]:
x_trn = training_impute_median_norm.drop(columns=['subject_id','slope_following_9_months'])
y_trn = training_impute_median_norm['slope_following_9_months']
x_tst = testing_impute_median_norm.drop(columns=['subject_id','slope_following_9_months'])
y_tst = testing_impute_median_norm['slope_following_9_months']
x_val = validation_impute_median_norm.drop(columns=['subject_id','slope_following_9_months'])
y_val = validation_impute_median_norm['slope_following_9_months']

In [6]:
data = training_impute_median_norm.append(testing_impute_median_norm)
X = x_trn.append(x_tst)
y = y_trn.append(y_tst)

# Evaluate Metrics

In [7]:
def evaluation_metrics(y_test, y_pred, x_trn, n):
  R2 = metrics.r2_score(y_test, y_pred).round(4)
  Adjusted_R2 = (1 - (1-metrics.r2_score(y_test, y_pred))*(len(y_test)-1)/(len(y_test)-x_trn.shape[1]-1)).round(4)
  MAE = metrics.mean_absolute_error(y_test, y_pred).round(4)
  MSE = metrics.mean_squared_error(y_test, y_pred).round(4)
  RMSE = np.sqrt(metrics.mean_squared_error(y_test, y_pred)).round(4)
  Pearson_Correlation = pearsonr(y_test, y_pred)[0].round(4)

  evaluatin_table = pd.DataFrame.from_dict({"feature": n,
                                            "R^2": R2,
                                            "Adjusted R^2": Adjusted_R2,
                                            "MAE": MAE,
                                            "MSE": MSE,
                                            "RMSE": RMSE,
                                            "Pearson_Correlation": [Pearson_Correlation],
                                            }, 
  )

  return evaluatin_table

# NN

In [8]:
x_trn.shape

(1710, 325)

In [9]:
# define the model
model_nn = Sequential()
model_nn.add(Dense(8, input_dim=325, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(16, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(32, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(64, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(64, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(32, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(16, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(8, kernel_initializer='normal', activation='relu'))
model_nn.add(Dense(1, kernel_initializer='normal'))
# Compile model
model_nn.compile(optimizer='adam',
                 loss='mean_squared_error')

In [10]:
model_nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 8)                 2608      
_________________________________________________________________
dense_1 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_3 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_4 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 16)                5

In [22]:
start_NN = time()
history = model_nn.fit(x_trn,
                       y_trn,
                       batch_size = 64,
                       epochs = 10,
                       validation_data=(x_tst, y_tst))
y_pred_train = model_nn.predict(x_trn).reshape(-1)
y_pred_test = model_nn.predict(x_tst).reshape(-1)
runtime_NN = np.round(time()-start_NN,5)
print(runtime_NN)
print(evaluation_metrics(y_tst, y_pred_test, x_tst,len(x_tst.columns)))

Epoch 1/10
27/27 [==============================] - 0s 13ms/step - loss: 0.4048 - val_loss: 0.4597
Epoch 2/10
27/27 [==============================] - 0s 5ms/step - loss: 0.3489 - val_loss: 0.4956
Epoch 3/10
27/27 [==============================] - 0s 6ms/step - loss: 0.3320 - val_loss: 0.5066
Epoch 4/10
27/27 [==============================] - 0s 6ms/step - loss: 0.3375 - val_loss: 0.5227
Epoch 5/10
27/27 [==============================] - 0s 6ms/step - loss: 0.3195 - val_loss: 0.5590
Epoch 6/10
27/27 [==============================] - 0s 5ms/step - loss: 0.3131 - val_loss: 0.5715
Epoch 7/10
27/27 [==============================] - 0s 6ms/step - loss: 0.3105 - val_loss: 0.6121
Epoch 8/10
27/27 [==============================] - 0s 5ms/step - loss: 0.3035 - val_loss: 0.5973
Epoch 9/10
27/27 [==============================] - 0s 5ms/step - loss: 0.2943 - val_loss: 0.6312
Epoch 10/10
27/27 [==============================] - 0s 5ms/step - loss: 0.2894 - val_loss: 0.6586
2.49932
   feature

In [26]:
start_NN = time()
history = model_nn.fit(X,
                       y,
                       batch_size = 64,
                       epochs = 10)
y_pred_val = model_nn.predict(x_val).reshape(-1)
runtime_NN = np.round(time()-start_NN,5)
print("NN", runtime_NN,"sec")
print(evaluation_metrics(y_val, y_pred_val, x_val,len(x_val.columns)))

Epoch 1/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2823
Epoch 2/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2851
Epoch 3/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2768
Epoch 4/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2855
Epoch 5/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2812
Epoch 6/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2847
Epoch 7/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2797
Epoch 8/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2766
Epoch 9/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2735
Epoch 10/10
34/34 [==============================] - 0s 4ms/step - loss: 0.2760
NN 1.83977 sec
   feature     R^2  Adjusted R^2     MAE     MSE    RMSE  Pearson_Correlation
0      325 -0.5761       -1.2806  0.8262  1.4629  1.2095               0.1678


# AutoKeras

In [13]:
# It tries 10 different models.
reg = ak.StructuredDataRegressor(max_trials=3, overwrite=True)
# Feed the structured data regressor with training data.
reg.fit(x_trn, y_trn, epochs=10)
# Predict with the best model.
y_pred = reg.predict(x_tst)
# Evaluate the best model with testing data.
print(reg.evaluate(x_tst, y_tst))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
structured_data...|True              |?                 
structured_data...|2                 |?                 
structured_data...|False             |?                 
structured_data...|0                 |?                 
structured_data...|32                |?                 
structured_data...|32                |?                 
regression_head...|0                 |?                 
optimizer         |adam              |?                 
learning_rate     |0.001             |?                 



KeyboardInterrupt: ignored

In [13]:
model = reg.export_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 325)]             0         
_________________________________________________________________
multi_category_encoding (Mul (None, 325)               0         
_________________________________________________________________
normalization (Normalization (None, 325)               651       
_________________________________________________________________
dense (Dense)                (None, 32)                10432     
_________________________________________________________________
re_lu (ReLU)                 (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 32)                0     

In [15]:
print(evaluation_metrics(y_tst, y_pred.reshape(-1), x_tst,len(x_trn.columns)))

   feature     R^2  Adjusted R^2     MAE     MSE    RMSE  Pearson_Correlation
0      325 -1.2156        -8.275  1.0068  1.5672  1.2519               0.2282
